In [1]:
from bs4 import BeautifulSoup
import urllib.request, urllib.error, urllib.parse
import pandas as pd
from selenium import webdriver

In [2]:
cnn_url = 'https://edition.cnn.com'
world_url = 'https://edition.cnn.com/world'
politics_url = 'https://edition.cnn.com/politics'
health_url = 'https://edition.cnn.com/health'
entertainment_url = 'https://edition.cnn.com/entertainment'

In [3]:
def getParseUrl(link):
    driver = getDriver()
    driver.get(link)
    webContent = driver.page_source
    
    data = BeautifulSoup(webContent, 'html.parser')
    driver.close()
    return data

In [4]:
def getParseUrlStatic(link):
    response = urllib.request.urlopen(link)
    webContent = response.read()
    
    data = BeautifulSoup(webContent, 'html.parser')
    return data

In [5]:
def getNewsLinks(data):
    headlines = data.find_all(class_ = "cd__headline")
    links = []
    for headline in headlines:
        link = headline.a["href"]
        if link[:5] != 'https':
            link = cnn_url + link
        links.append(link)
    return links

In [6]:
def getDriver():
    driver = webdriver.Chrome(executable_path=r'C:\Users\DELL\Downloads\chromedriver_win32_5\chromedriver')
    return driver

In [7]:
def getStringFromTags(tag):
    data = tag.contents
    string = ""
    for content in data:
        try:
            string += content
        except:
            string += getStringFromTags(content)
    return string

In [8]:
def getNewsFromLink(link, genre):
    news = {}
    try:
        data = getParseUrlStatic(link)
        news['title'] = data.h1.contents[0]
        news['summary'] = ""
        divs = data.find_all(True, {'class': ["zn-body__paragraph", "Paragraph__component BasicArticle__paragraph BasicArticle__pad", "Paragraph__component"]})
        for div in divs:
            news['summary'] += getStringFromTags(div)
        news['image_link'] = data.find('img')['data-src-mini']
        news['link'] = link
        news['genre'] = genre
        if genre == 'world':
            news['genre'] = 'politics'
        if news['summary'] == '':
            news = {}
        news['summary'] = news['summary'].split('(CNN)', 1)[1]
    except:
        news = {}
        print("Cannot get this news " + link)
    return news

In [9]:
def getDataFromUrl(url, genre):
    data = getParseUrl(url)
    links = getNewsLinks(data)
    news = [getNewsFromLink(link, genre) for link in links if link[:23] == 'https://edition.cnn.com']
    return news

In [10]:
news = []
for genre in ['world', 'politics', 'entertainment']:
    url = cnn_url + '/' + genre
    print(genre)
    cur_news = getDataFromUrl(url, genre)
    for new in cur_news:
        news.append(new)

world
Cannot get this news https://edition.cnn.com/2021/11/06/middleeast/iraq-prime-minister-drone-attack-intl-hnk/index.html
Cannot get this news https://edition.cnn.com/videos/style/2021/11/05/benin-bronzes-nigeria-mclean-dnt-oneworld-intl-vpx.cnn
Cannot get this news https://edition.cnn.com/2021/11/05/europe/cop26-youth-protests-friday-intl/index.html
Cannot get this news https://edition.cnn.com/2021/11/03/middleeast/auschwitz-tattoo-kit-auction-scli-intl/index.html
Cannot get this news https://edition.cnn.com/2021/11/03/middleeast/marib-yemen-houthis-intl/index.html
Cannot get this news https://edition.cnn.com/2021/11/02/middleeast/israel-cop26-wheelchair-access-intl/index.html
Cannot get this news https://edition.cnn.com/2021/11/06/africa/ethiopia-military-volunteers-rebel-advance-intl/index.html
Cannot get this news https://edition.cnn.com/2021/11/07/americas/nicaragua-election-nov-7-intl-latam/index.html
Cannot get this news https://edition.cnn.com/2021/11/04/asia/indonesia-defo

Cannot get this news https://edition.cnn.com/2021/11/05/world/youth-activists-climate-change-cop26/index.html
Cannot get this news https://edition.cnn.com/2021/11/05/world/gallery/augmented-reality-extinct-animals-revival-spc-intl-scn/index.html
Cannot get this news https://edition.cnn.com/2021/11/05/china/hypersonic-weapons-china-us-russia-intl-cmd/index.html
Cannot get this news https://edition.cnn.com/2021/11/04/asia/indonesia-deforestation-cop26-pledge-intl-hnk/index.html
Cannot get this news https://edition.cnn.com/2021/11/04/asia/submarine-uss-connecticut-accident-undersea-mountain-hnk-intl-ml-dst/index.html
Cannot get this news https://edition.cnn.com/2021/11/04/asia/north-korea-uranium-report-intl-hnk/index.html
Cannot get this news https://edition.cnn.com/2021/11/04/world/cop26-day-4-takeaways-intl-climate/index.html
politics
Cannot get this news https://edition.cnn.com/2021/11/07/politics/jennifer-granholm-winter-heating-costs-cnntv/index.html
Cannot get this news https://edi

In [11]:
len(news)

172

In [13]:
pd.DataFrame(news).dropna()

,title,summary,image_link,link,genre
0,"Koalas are dying from chlamydia, and climate c...",A silent killer is spreading through Australia...,//cdn.cnn.com/cnnnext/dam/assets/211104215814-...,https://edition.cnn.com/2021/11/06/australia/a...,politics
3,Death of pregnant woman ignites debate about a...,The death of a pregnant Polish woman has reign...,//cdn.cnn.com/cnnnext/dam/assets/211107090155-...,https://edition.cnn.com/2021/11/07/europe/pola...,politics
4,A dangerous crisis is brewing in the Balkans. ...,The Balkan state of Bosnia-Herzegovina is on t...,//cdn.cnn.com/cnnnext/dam/assets/211105150200-...,https://edition.cnn.com/2021/11/06/europe/bosn...,politics
5,Camels escape from circus and venture out to e...,Spanish police had an unusual encounter on Fri...,//cdn.cnn.com/cnnnext/dam/assets/211106062928-...,https://edition.cnn.com/2021/11/06/europe/esca...,politics
7,Russian diplomat found dead on street outside ...,A Russian diplomat found dead on the sidewalk ...,//cdn.cnn.com/cnnnext/dam/assets/211105102616-...,https://edition.cnn.com/2021/11/05/europe/germ...,politics
...,...,...,...,...,...
164,'Colin in Black & White' explores Colin Kaeper...,Colin Kaepernick possesses greater gifts as an...,//cdn.cnn.com/cnnnext/dam/assets/211025215218-...,https://edition.cnn.com/2021/10/27/entertainme...,entertainment
165,'Queens' gets the band back together for a soa...,"""Queens"" is an extremely familiar idea, elevat...",//cdn.cnn.com/cnnnext/dam/assets/210902034712-...,https://edition.cnn.com/2021/10/19/entertainme...,entertainment
166,'Succession' doesn't miss a beat as its Murdoc...,"Success hasn't spoiled ""Succession,"" as the Em...",//cdn.cnn.com/cnnnext/dam/assets/211012181811-...,https://edition.cnn.com/2021/10/15/entertainme...,entertainment
167,'The Wonder Years' shines in ABC's reboot of t...,Early '90s TV is hitting the refresh button th...,//cdn.cnn.com/cnnnext/dam/assets/210902034520-...,https://edition.cnn.com/2021/09/22/entertainme...,entertainment
